In [1]:
import os
from config import *
import torch
from dataloaders.crnn_dataloader import *
from torch.utils.data import random_split, DataLoader
import pandas as pd
import polars as pl
from torch.masked import masked_tensor, as_masked_tensor


In [2]:
float("-inf")

-inf

In [2]:
train_quick = pl.read_parquet(P_TRAIN_PARQUET_QUICK)

In [4]:
train_quick.columns[-2]

''

## Recomended Approach: USE parquets

In [2]:
parquet_dataset = ParquetCRNNDataset(P_TRAIN_PARQUET_QUICK)

In [3]:
parquet_dataset.sequence_df.row(0)

('GGGAACGACUCGAGUAGAGUCGAAAAGACCAUGAGGCCUUGACCGAUCGCGAGUUGGCCUGGUCGCAGACCGACUUCCCAAGCGAAGCUCCGGCGCGUUUCGAGGGCGACUCGUAUAACGAAGGCGGGUAGAGACGGCAUUCGUGCCGUCUCUACCAAAAGAAACAACAACAACAAC',
 'DATA\\Ribonanza_bpp_files\\extra_data\\5\\8\\b\\0ab0ec3d057c.txt',
 177)

In [4]:
generator1 = torch.Generator().manual_seed(42)
train_dataset, val_dataset = random_split(parquet_dataset, [0.7, 0.3], generator1)

In [5]:
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, persistent_workers=True, pin_memory=True)  
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4, persistent_workers=True, pin_memory=True) 

In [6]:
len(val_dataloader)

1647

In [7]:
x, y , m = train_dataset.__getitem__(17)

In [8]:
y.shape

torch.Size([512])

In [ ]:
for i in range(len(train_dataset)):
    x, y, m = train_dataset.__getitem__(i)  
    if y.shape[0] != 512:
        print(i, y.shape) 
        break
                     

In [6]:
for x, y , z in train_dataloader:
    print(y.shape)
    break

torch.Size([16, 512])


In [6]:
test_dataset = InferenceParquetCRNNDataset(P_TEST_PARQUET, train_test_flag='test')

In [ ]:
train_dataset.__getitem__(10)

In [8]:
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, persistent_workers=True, pin_memory=True)

#### OPTTIONAL: use pandas 
uncomment if necessary


In [10]:
# train = pd.read_csv(P_TRAIN_CSV)

In [11]:
#dataset = CNNDataset(data_csv=P_TRAIN_CSV, target_csv=P_TARGETS_CSV, matrix_dir=ETERNA_PKG_BPP )

In [12]:
#dataset.matrix_df[dataset.matrix_df['sequence_id'] == 'eee73c1836bc']['path']

In [13]:
# generator1 = torch.Generator().manual_seed(42)
# train_dataset, val_dataset = random_split(parquet_dataset, [0.7, 0.3], generator1)

In [14]:
# train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, persistent_workers=True)  
# val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4, persistent_workers=True) 

In [15]:

# test_dataset = CNNDataset(data_csv=P_TEST_CSV, train_test_flag='test', matrix_dir=ETERNA_PKG_BPP)

In [16]:
# test_dataloader = DataLoader(test_dataset, batch_size=512, shuffle=False, num_workers=8, persistent_workers=True, pin_memory=True)

In [17]:

# test_dataset.matrix_df[test_dataset.matrix_df['sequence_id'] == 'eee73c1836bc']['path'].values[0]
#test_dataset.matrix_df

In [18]:
# for x, y in train_dataloader:
#     print(x.shape)
#     break

In [19]:
# test_dataset.__getitem__(0)

In [20]:
# import torch
# import torch.nn.functional as F

# import numpy as np

# import os
# from tqdm import tqdm
# import torch
# from torch.utils.data import DataLoader, random_split
# import torch.optim as optim
# from torch.nn import CTCLoss

# from dataloaders import CNNDataset
# from config import *


# # 📉 Define loss functions for training and evaluation
# def loss_fn(output, target):
#     # 🪟 Clip the target values to be within the range [0, 1]
#     clipped_target = torch.clip(target, min=0, max=1)
#     # 📉 Calculate the mean squared error loss
#     mses = F.l1_loss(output, clipped_target, reduction='mean')
#     return mses

# # def mae_fn(output, target):
# #     # 🪟 Clip the target values to be within the range [0, 1]
# #     clipped_target = torch.clip(target, min=0, max=1)
# #     # 📉 Calculate the mean absolute error loss
# #     maes = F.l1_loss(output, clipped_target, reduction='mean')
# #     return maes

# def train_batch(crnn, data, optimizer, criterion, device):
#     #crnn.train()

#     images, targets= [d.to(device) for d in data]

#     logits = crnn(images)
#     #log_probs = torch.nn.functional.log_softmax(logits, dim=2)
#     log_probs = logits

#     #atch_size = images.size(0)
#     #input_lengths = torch.LongTensor([logits.size(0)] * batch_size)
#     #target_lengths = torch.flatten(input_lengths)

#     loss = criterion(log_probs, targets)

#     optimizer.zero_grad()
#     loss.backward()
#     torch.nn.utils.clip_grad_norm_(crnn.parameters(), 5) # gradient clipping with 5
#     optimizer.step()
#     return loss.item()


# def main():
#     epochs = 1
#     #dataset = CNNDataset(data_csv=P_TRAIN_CSV, target_csv=P_TARGETS_CSV, matrix_dir=ETERNA_PKG_BPP )
#     generator1 = torch.Generator().manual_seed(42)
#     #train_dataset, val_dataset = random_split(dataset, [0.7, 0.3], generator1)
#     #dataset = CNNDataset(data_csv=P_TRAIN_CSV, target_csv=P_TARGETS_CSV, matrix_dir=ETERNA_PKG_BPP )
#     #train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, persistent_workers=True)  
#     #val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4, persistent_workers=True) 
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#     model = CRNN(img_channel=1, img_width=224, img_height=224)
#     print(model)
#     #if reload_checkpoint:
#     #    crnn.load_state_dict(torch.load(reload_checkpoint, map_location=device))
#     model.to(device)

#     # 📈 Define the optimizer with learning rate and weight decay
#     optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=5e-4)

#     # 🚂 Iterate over epochs
#     for epoch in range(epochs):
#         train_losses = []
#         model.train()
        
#         # 🚞 Iterate over batches in the training dataloader
#         for batch in (pbar := tqdm(train_dataloader, position=0, leave=True)):
#             loss = train_batch(model, batch, optimizer, loss_fn, device)
#             train_losses.append(loss.detach().cpu().numpy())
#             pbar.set_description(f"Train loss {loss.detach().cpu().numpy():.4f}")
        
#         # 📊 Print average training loss and MAE for the epoch
#         print(f"Epoch {epoch} train loss: ", np.mean(train_losses))
        
#         val_losses = []
#         model.eval()
        
#         # 🚞 Iterate over batches in the validation dataloader
#         for batch in (pbar := tqdm(val_dataloader, position=0, leave=True)):
#             loss = train_batch(model, batch, optimizer, loss_fn, device)
#             optimizer.zero_grad()
#             out = model(batch.x, batch.edge_index)
#             out = torch.squeeze(out)
#             val_losses.append(loss.detach().cpu().numpy())
#             pbar.set_description(f"Validation loss {loss.detach().cpu().numpy():.4f}")
        
#         # 📊 Print average validation loss and MAE for the epoch
#         print(f"Epoch {epoch} val loss: ", np.mean(val_losses))
#         print(f"Epoch {epoch} val mae: ", np.mean(val_maes))


In [21]:
# targets.iloc[0, :-1].values.astype('float32').shape

In [22]:
# test = 'DATA/Ribonanza_bpp_files/extra_data/9/8/2/ff4dcd9bf671.txt'
# import numpy as np

# # Load the data from the file
# data = np.loadtxt(test)

# # Determine the shape of the array
# max_row = int(data[:, 0].max())
# max_col = int(data[:, 1].max())

# # Create an empty array filled with zeros
# filled_array = np.zeros((max_row, max_col))

# # Fill the values from the loaded data into the empty array
# for row, col, value in data:
#     filled_array[int(row) - 1, int(col) - 1] = value

In [6]:

import torch
from torchvision import models
import pytorch_lightning as pl
from pytorch_lightning.utilities.types import STEP_OUTPUT
from torchvision import models
from torchmetrics.classification import F1Score, AUROC, Recall, ROC, Accuracy, Precision, Specificity 

In [7]:
import torch.nn as nn


class CRNN(nn.Module):

    def __init__(self, img_channel, img_height, img_width,
                 map_to_seq_hidden=64, rnn_hidden=256, leaky_relu=False):
        super(CRNN, self).__init__()

        self.cnn, (output_channel, output_height, output_width) = \
            self._cnn_backbone(img_channel, img_height, img_width, leaky_relu)

        self.map_to_seq = nn.Linear(output_channel * output_height, map_to_seq_hidden)

        self.rnn1 = nn.LSTM(map_to_seq_hidden, rnn_hidden, bidirectional=True)
        self.rnn2 = nn.LSTM(2 * rnn_hidden, rnn_hidden, bidirectional=True)

        self.dense = nn.Linear(2 * rnn_hidden, 1)
  
       

    def _cnn_backbone(self, img_channel, img_height, img_width, leaky_relu):
        assert img_height % 16 == 0
        assert img_width % 4 == 0

        channels = [img_channel, 64, 128, 256, 256, 512, 512, 512]
        kernel_sizes = [3, 3, 3, 3, 3, 3, 3]
        strides = [1, 1, 1, 1, 1, 1, 1]
        paddings = [1, 1, 1, 1, 1, 1, 1]

        cnn = nn.Sequential()

        def conv_relu(i, batch_norm=False):
            # shape of input: (batch, input_channel, height, width)
            input_channel = channels[i]
            output_channel = channels[i+1]

            cnn.add_module(
                f'conv{i}',
                nn.Conv2d(input_channel, output_channel, kernel_sizes[i], strides[i], paddings[i])
            )

            if batch_norm:
                cnn.add_module(f'batchnorm{i}', nn.BatchNorm2d(output_channel))

            relu = nn.LeakyReLU(0.2, inplace=True) if leaky_relu else nn.ReLU(inplace=True)
            cnn.add_module(f'relu{i}', relu)

        # size of image: (channel, height, width) = (img_channel, img_height, img_width)
        conv_relu(0)
        cnn.add_module('pooling0', nn.MaxPool2d(kernel_size=(2,1), stride=(2,1)))
        # (64, img_height // 2, img_width // 2)

        conv_relu(1)
        cnn.add_module('pooling1', nn.MaxPool2d(kernel_size=(2,1), stride=(2,1)))
        # (128, img_height // 4, img_width // 4)

        conv_relu(2)
        conv_relu(3)
        cnn.add_module(
            'pooling2',
            nn.MaxPool2d(kernel_size=(2, 1))
        )  # (256, img_height // 8, img_width // 4)

        conv_relu(4, batch_norm=True)
        conv_relu(5, batch_norm=True)
        cnn.add_module(
            'pooling3',
            nn.MaxPool2d(kernel_size=(2, 1))
        )  # (512, img_height // 16, img_width // 4)

        conv_relu(6)  # (512, img_height // 16 - 1, img_width // 4 - 1)

        output_channel, output_height, output_width = \
            channels[-1], img_height // 16 , img_width // 4 
        print((output_channel, output_height, output_width))
        return cnn, (output_channel, output_height, output_width)

    def forward(self, images):
        # shape of images: (batch, channel, height, width)
        conv = self.cnn(images)
        
        # shape of conv: (batch, channel, height, width)
        batch, channel, height, width = conv.size()
        conv = conv.view(batch, channel * height, width)
        conv = conv.permute(2, 0, 1)  # (width, batch, feature)
        seq = self.map_to_seq(conv)
        
        recurrent, _ = self.rnn1(seq)
        recurrent, _ = self.rnn2(recurrent)

        output = self.dense(recurrent)
        # reshape to batch, seq_length
        output = output.transpose(0, 1)
        return output  # shape: (seq_len, batch, num_class)

In [8]:
class CNNTrainer(pl.LightningModule):
    def __init__(
        self,
        model_name,
        weights=None,
        learning_rate=0.001,
        weight_decay=1e-4,
        gamma=2,
        num_classes = 223,
        num_channels = 1,
    ):
        """Initialize with pretrained weights instead of None if a pretrained model is required."""
        super().__init__()
        self.training_step_losses = []
        self.training_step_accuracy = []

        self.validation_step_losses = []
        self.validation_step_accuracy = []

        self.test_step_losses = []

        self.weights = weights
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.gamma = gamma

        if num_classes == 2:
            self.num_classes = num_classes - 1
        elif num_classes > 2:
            self.num_classes = num_classes

        self.num_channels = num_channels

        self.model_name = model_name

        self.conv_reshape = []
        # models ###################
        if self.model_name == "crnn":
            self.model = self._get_model()
     
        # possible bug with cross entropy and two classes


        # metrics ###################
        # self.f1 = F1Score(task=self.classification, num_classes=self.num_classes)
        # self.auroc = AUROC(task=self.classification, num_classes=self.num_classes)
        # self.recall = Recall(task=self.classification, num_classes=self.num_classes)
        # self.accuracy = Accuracy(task=, num_classes=self.num_classes)
        # self.precision = Precision(task=self.classification, num_classes=self.num_classes)
        # self.specifity = Specificity(task=self.classification, num_classes=self.num_classes)
        self.save_hyperparameters()

    def loss_fn(self, output, target):
        # 🪟 Clip the target values to be within the range [0, 1]
        #clipped_target = torch.clip(target, min=0, max=1)
        # 📉 Calculate the mean squared error loss
        
        #mses = torch.nn.functional.l1_loss(output, clipped_target, reduction='mean')
        mses = torch.nn.functional.l1_loss(output, target, reduction='mean')

        return mses

        
    def _get_model(self):
        model = CRNN(img_channel=1, img_width=512, img_height=512)

        return model


    # def _metrics(self, y_pred, y_true):
    #     f1 = self.f1(y_pred, y_true)
    #     auroc = self.auroc(y_pred, y_true)
    #     recall = self.recall(y_pred, y_true)
    #     precision = self.precision(y_pred, y_true)
    #     accuracy = self.accuracy(y_pred, y_true)
    #     specifity = self.specifity(y_pred, y_true)
    #     return accuracy

    def forward(self, imgs):
        output = self.model(imgs)
        #print(output)
        return output
    
    
    def configure_optimizers(self):
       
        optimizer = torch.optim.Adam(
            self.model.parameters(),
            lr=self.learning_rate,
            weight_decay=self.weight_decay
        )
        return optimizer

    def training_step(self, batch, batch_idx):
        inputs, y_true, mask = batch
        

        y_pred = self.forward(inputs)
        y_pred = y_pred * mask.unsqueeze(-1)
        #print(mask.unique())
        #print(y_pred.unique())       
        # loss
    
        y_true = y_true.unsqueeze(-1)
        y_true = y_true * mask.unsqueeze(-1) 
        loss = self.loss_fn(y_pred, y_true)

        # accuracy = self._metrics(y_pred, y_true)

        self.training_step_losses.append(loss)
        # self.training_step_accuracy.append(accuracy)

        self.log("train_loss", loss, prog_bar=True, on_step=True)
        # self.log("train_accuracy", accuracy, prog_bar=True, on_step=True)
        # self.log("train_auroc", auroc, prog_bar=False, on_step=True)
        # self.log("train_precision", precision, prog_bar=False, on_step=True)
        # self.log("train_recall", recall, prog_bar=False, on_step=True)
        # self.log("train_f1", f1, prog_bar=False, on_step=True)
        # self.log("train_specifity", specifity, prog_bar=False, on_step=True)


        return loss

    def on_train_epoch_end(self):
        avg_loss = torch.stack(self.training_step_losses).mean()
        # avg_train_acc = torch.stack(self.training_step_accuracy).mean()

        self.log(
            "train_loss_epoch_end",
            avg_loss,
            prog_bar=True,
            on_step=False,
            on_epoch=True,
        )
        # self.log(
        #     "train_acc_epoch_end",
        #     avg_train_acc,
        #     prog_bar=True,
        #     on_step=False,
        #     on_epoch=True,
        # )
        return avg_loss

    def validation_step(self, batch, batch_idx):
        inputs, y_true, mask = batch

        y_pred = self.forward(inputs)
        y_pred = y_pred * mask.unsqueeze(-1)
     
    
        # loss
        y_true = y_true.unsqueeze(-1)
        y_true = y_true * mask.unsqueeze(-1) 
        loss = self.loss_fn(y_pred, y_true)

        # accuracy = self._metrics(y_pred, y_true)

        self.validation_step_losses.append(loss)
        # self.validation_step_accuracy.append(accuracy)

        self.log("val_loss", loss, prog_bar=True, on_step=True)
        # self.log("val_accuracy", accuracy, prog_bar=True, on_step=True)
        # self.log("val_auroc", auroc, prog_bar=False, on_step=True)
        # self.log("val_precision", precision, prog_bar=False, on_step=True)
        # self.log("val_recall", recall, prog_bar=False, on_step=True)
        # self.log("val_f1", f1, prog_bar=False, on_step=True)
        # self.log("val_specifity", specifity, prog_bar=False, on_step=True)


        return loss

    def on_validation_epoch_end(self):
        avg_loss = torch.stack(self.validation_step_losses).mean()
        # avg_train_acc = torch.stack(self.validation_step_accuracy).mean()

        self.log(
            "validation_loss_epoch_end",
            avg_loss,
            prog_bar=True,
            on_step=False,
            on_epoch=True,
        )
        # self.log(
        #     "validation_acc_epoch_end",
        #     avg_train_acc,
        #     prog_bar=True,
        #     on_step=False,
        #     on_epoch=True,
        # )
        return avg_loss

    # This is for binaryCE loss cuz one dimension 

    def predict_step(self, batch, batch_idx):
        inputs, mask = batch
        y_pred = self.forward(inputs)
        y_pred = y_pred * mask.unsqueeze(-1)
        # if self.classification == 'binary':
        #     y_pred = torch.nn.functional.sigmoid(y_pred)
        return y_pred

In [13]:
model = CNNTrainer("crnn")

(512, 32, 128)


In [12]:
from torchinfo import summary
print(summary(model, (64, 1, 512, 512)))

Layer (type:depth-idx)                   Output Shape              Param #
CNNTrainer                               [64, 512, 1]              --
├─CRNN: 1-1                              [64, 512, 1]              --
│    └─Sequential: 2-1                   [64, 512, 32, 512]        --
│    │    └─Conv2d: 3-1                  [64, 64, 512, 512]        640
│    │    └─ReLU: 3-2                    [64, 64, 512, 512]        --
│    │    └─MaxPool2d: 3-3               [64, 64, 256, 512]        --
│    │    └─Conv2d: 3-4                  [64, 128, 256, 512]       73,856
│    │    └─ReLU: 3-5                    [64, 128, 256, 512]       --
│    │    └─MaxPool2d: 3-6               [64, 128, 128, 512]       --
│    │    └─Conv2d: 3-7                  [64, 256, 128, 512]       295,168
│    │    └─ReLU: 3-8                    [64, 256, 128, 512]       --
│    │    └─Conv2d: 3-9                  [64, 256, 128, 512]       590,080
│    │    └─ReLU: 3-10                   [64, 256, 128, 512]       --


In [14]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from config import *

checkpoint_callback = ModelCheckpoint(
                dirpath=CRNN_CHK_PNT,
                monitor='val_loss',
                save_top_k=1,
                filename='best-{epoch}-{val_loss:.2f}'
            )

logger = TensorBoardLogger(save_dir=CRNN_LOG, name='1epoch_crnn', version=1)

pl_trainer = Trainer(max_epochs = 1, accelerator='gpu', devices=1, precision="16-mixed", default_root_dir=CRNN_CHK_PNT)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [15]:
pl_trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | CRNN | 10.1 M
-------------------------------
10.1 M    Trainable params
0         Non-trainable params
10.1 M    Total params
40.589    Total estimated model params size (MB)


Epoch 0: 100%|██████████| 7685/7685 [51:20<00:00,  2.49it/s, v_num=1, train_loss=0.0749, val_loss_step=0.0643, val_loss_epoch=0.0729, validation_loss_epoch_end=0.0729]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 7685/7685 [51:20<00:00,  2.49it/s, v_num=1, train_loss=0.0749, val_loss_step=0.0643, val_loss_epoch=0.0729, validation_loss_epoch_end=0.0729]


In [ ]:
#Reload your desired checkpoint
#model = CNNTrainer.load_from_checkpoint(r"E:\Studies\MAT6493\project\RNA_Reactivity_Prediction\experiments\crnn\lightning_logs\version_1\checkpoints\epoch=0-step=7685.ckpt")
y_pred = pl_trainer.predict(model=model, dataloaders=test_dataloader)

In [63]:
rand = torch.arange(16*3).reshape(4, 4, 3)
mask = torch.where(rand % 2 == 0)
rand[mask]

tensor([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34,
        36, 38, 40, 42, 44, 46])

### GNN BASELINE

In [1]:
from dataloaders.edgecnn_dataloader import *
from trainers.gnn_trainer import *
import os
import torch
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader
import pandas as pd
import pytorch_lightning as pl
from config import *
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from tqdm import tqdm

In [2]:
train_val_dataset = SimpleGraphDataset(P_TRAIN_PARQUET_QUICK, edge_distance=4)
test_dataset = InferenceGraphDataset(P_TEST_PARQUET, edge_distance=4)

In [3]:
train_val_dataset.num_features

4

In [4]:
generator1 = torch.Generator().manual_seed(42)
train_dataset, val_dataset = random_split(train_val_dataset, [0.7, 0.3], generator1)

In [5]:
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, persistent_workers=True, pin_memory=True)  
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4, persistent_workers=True, pin_memory=True)


In [6]:
train_dataset.__getitem__(0)

Data(x=[177, 4], edge_index=[2, 1396], y=[177], valid_mask=[100])

In [7]:
#model = GNNTrainer("edgecnn", num_features=train_val_dataset.num_features, num_channels=4)

In [9]:
checkpoint_callback = ModelCheckpoint(
            dirpath=EDGECNN_CHK_PNT,
            monitor='val_loss',
            save_top_k=1,
            filename='best-{epoch}-{val_loss:.2f}'
        )

#logger = TensorBoardLogger(save_dir=log, name=f"{args.num_epoch}epoch_{args.model_name}", version=1)
logger = TensorBoardLogger(save_dir=EDGECNN_LOG, name=f"1epoch_crnn", version=1)



# fit the model
pl_trainer = pl.Trainer(max_epochs = 1, accelerator='cpu', devices=1, default_root_dir=EDGECNN_CHK_PNT)
#pl_trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)
model = GNNTrainer.load_from_checkpoint("./experiments/edgecnn/lightning_logs/version_37/checkpoints/epoch=0-step=11135.ckpt")


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/mina/miniconda3/envs/main-env/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


In [10]:
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4, persistent_workers=True, pin_memory=True)

In [12]:
predictions = pl_trainer.predict(model=model, dataloaders=test_dataloader)

Predicting DataLoader 0:   1%|          | 1042/86344 [00:46<1:04:07, 22.17it/s]
Predicting: |          | 83192/? [07:58<00:00, 173.98it/s]

/Users/mina/miniconda3/envs/main-env/lib/python3.11/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [16]:
from torch_geometric.nn.models import EdgeCNN
device = 'cpu'

# 🛠️ Set the device to GPU if available, otherwise use CPU
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 🏗️ Initialize the EdgeCNN model with specified parameters
model = EdgeCNN(
    in_channels=train_val_dataset.num_features,  # 📊 Input features determined by the dataset
    hidden_channels=128,  # 🕳️ Number of hidden channels in the model
    num_layers=4,  # 🧱 Number of layers in the model
    out_channels=1  # 📤 Number of output channels
).to(device)  # 🏗️ Move the model to the selected device (GPU or CPU)

In [17]:
# 📉 Define loss functions for training and evaluation
import torch.nn.functional as F

def loss_fn(output, target):
    # 🪟 Clip the target values to be within the range [0, 1]
    clipped_target = torch.clip(target, min=0, max=1)
    # 📉 Calculate the mean squared error loss
    mses = F.mse_loss(output, clipped_target, reduction='mean')
    return mses

def mae_fn(output, target):
    # 🪟 Clip the target values to be within the range [0, 1]
    clipped_target = torch.clip(target, min=0, max=1)
    # 📉 Calculate the mean absolute error loss
    maes = F.l1_loss(output, clipped_target, reduction='mean')
    return maes

In [18]:
n_epochs = 1

# 📈 Define the optimizer with learning rate and weight decay
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=5e-4)

# 🚂 Iterate over epochs
for epoch in range(n_epochs):
    train_losses = []
    train_maes = []
    model.train()
    
    # 🚞 Iterate over batches in the training dataloader
    for batch in (pbar := tqdm(train_dataloader, position=0, leave=True)):
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        out = torch.squeeze(out)
        loss = loss_fn(out[batch.valid_mask], batch.y[batch.valid_mask])
        mae = mae_fn(out[batch.valid_mask], batch.y[batch.valid_mask])
        loss.backward()
        train_losses.append(loss.detach().cpu().numpy())
        train_maes.append(mae.detach().cpu().numpy())
        optimizer.step()
        pbar.set_description(f"Train loss {loss.detach().cpu().numpy():.4f}")
    
    # 📊 Print average training loss and MAE for the epoch
    print(f"Epoch {epoch} train loss: ", np.mean(train_losses))
    print(f"Epoch {epoch} train mae: ", np.mean(train_maes))
    
    val_losses = []
    val_maes = []
    model.eval()
    
    # 🚞 Iterate over batches in the validation dataloader
    for batch in (pbar := tqdm(val_dataloader, position=0, leave=True)):
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        out = torch.squeeze(out)
        loss = loss_fn(out[batch.valid_mask], batch.y[batch.valid_mask])
        mae = mae_fn(out[batch.valid_mask], batch.y[batch.valid_mask])
        val_losses.append(loss.detach().cpu().numpy())
        val_maes.append(mae.detach().cpu().numpy())
        pbar.set_description(f"Validation loss {loss.detach().cpu().numpy():.4f}")
    
    # 📊 Print average validation loss and MAE for the epoch
    print(f"Epoch {epoch} val loss: ", np.mean(val_losses))
    print(f"Epoch {epoch} val mae: ", np.mean(val_maes))

Train loss 0.1454:   1%|          | 97/11135 [00:10<20:45,  8.86it/s]


KeyboardInterrupt: 